In [1]:
import aiohttp
import asyncio
import pandas as pd
from datetime import datetime
import copy

In [2]:
async def fetch_data_bybit(session, token, start_time, end_time):
    url = 'https://api.bybit.com/v5/market/kline'
    params = {
        'category': 'linear',
        'symbol': token,
        'interval': '1',
        'start': start_time,
        'end': end_time,
        'limit': 1000
    }
    async with session.get(url, params=params) as response:
        result = await response.json()
        return result['result']['list']

async def process_token_bybit(session, token, start_time, end_time):
    time_intervals = [start_time]
    cur_time = start_time
    while cur_time < end_time:
        time_intervals.append(cur_time + 1000 * 60 * 1000)
        cur_time = time_intervals[-1]
    
    frames = []
    for i in range(len(time_intervals) - 1):
        result = await fetch_data_bybit(session, token, time_intervals[i], time_intervals[i+1])
        cur_df = pd.DataFrame(result)
        cur_df.rename(columns={0: 'time', 1: 'open', 2: 'high', 3: 'low', 4: 'close', 5: 'volume', 6: 'turnover'}, inplace=True)
        frames.append(cur_df)
        
    all_frame = pd.concat(frames, ignore_index=True)
    all_frame.sort_values(by='time', inplace=True)
    all_frame.reset_index(drop=True, inplace=True)
    all_frame[['open', 'high', 'low', 'close', 'turnover']] = all_frame[['open', 'high', 'low', 'close', 'turnover']].astype('float')
    all_frame['token'] = token

    result_frame = all_frame
    condition1 = (result_frame.high / result_frame.low)
    condition1 = list(condition1[condition1 >= 1.015].index)
    
    condition2 = abs((result_frame.open / result_frame.close) - 1)
    condition2 = list(condition2[condition2 < 0.007].index)
    
    condition3 = abs((result_frame.close / result_frame.low))
    condition3 = list(condition3[condition3 >= 1.01].index)

    condition4 = abs((result_frame.high / result_frame.open))
    condition4 = list(condition4[condition4 >= 1.01].index)

    condition5 = list(result_frame[result_frame.turnover > 2 * 100000].index)
    
    cond1235 = set.intersection(set(condition1), set(condition2), set(condition3), set(condition5))
    cond1245 = set.intersection(set(condition1), set(condition2), set(condition4), set(condition5))

    check_frame = copy.deepcopy(result_frame.loc[sorted(list(cond1235) + list(cond1245))])
    check_frame['normal_time'] = [datetime.fromtimestamp(int(t) / 1000.0) for t in check_frame.time]

    if len(check_frame) > 0:
        return token, check_frame
    return None, None

async def get_token_outliear_bybit(tokens, start_time=1722508665488, end_time=1722632228542):
    check_tokens = []
    return_dict = {}

    async with aiohttp.ClientSession() as session:
        tasks = [process_token_bybit(session, token, start_time, end_time) for token in tokens]
        results = await asyncio.gather(*tasks)

    for token, check_frame in results:
        if token:
            check_tokens.append(token)
            return_dict[token] = check_frame

    return return_dict

In [3]:
return_dict = await get_token_outliear_bybit(tokens = ['POPCATUSDT', '1000000MOGUSDT'])

In [4]:
return_dict.keys()

dict_keys(['POPCATUSDT', '1000000MOGUSDT'])

In [5]:
return_dict['POPCATUSDT']

,time,open,high,low,close,volume,turnover,token,normal_time
520,1722539880000,0.6659,0.6754,0.6561,0.6637,1920275,1.282206e+06,POPCATUSDT,2024-08-01 21:18:00
520,1722539880000,0.6659,0.6754,0.6561,0.6637,1920275,1.282206e+06,POPCATUSDT,2024-08-01 21:18:00
1552,1722601800000,0.6841,0.6888,0.6773,0.6856,1030072,7.027272e+05,POPCATUSDT,2024-08-02 14:30:00
1697,1722610500000,0.6326,0.6369,0.6238,0.6303,1571625,9.896819e+05,POPCATUSDT,2024-08-02 16:55:00
1698,1722610560000,0.6303,0.6392,0.6288,0.6343,707788,4.497991e+05,POPCATUSDT,2024-08-02 16:56:00
2297,1722646500000,0.5342,0.5397,0.5312,0.5336,828798,4.434627e+05,POPCATUSDT,2024-08-03 02:55:00
2344,1722649320000,0.5309,0.5332,0.5231,0.5301,1128481,5.962684e+05,POPCATUSDT,2024-08-03 03:42:00


In [6]:
return_dict['1000000MOGUSDT']

,time,open,high,low,close,volume,turnover,token,normal_time
310,1722527280000,1.6110,1.6199,1.5955,1.6173,241025,386893.2197,1000000MOGUSDT,2024-08-01 17:48:00
313,1722527460000,1.6260,1.6271,1.5990,1.6219,136559,220167.7677,1000000MOGUSDT,2024-08-01 17:51:00
520,1722539880000,1.6510,1.6579,1.6171,1.6578,439502,717124.1056,1000000MOGUSDT,2024-08-01 21:18:00
1552,1722601800000,1.5917,1.6183,1.5886,1.6027,165473,264820.0952,1000000MOGUSDT,2024-08-02 14:30:00
1660,1722608280000,1.4966,1.5008,1.4691,1.4867,245593,364006.6982,1000000MOGUSDT,2024-08-02 16:18:00
